In [1]:
import cv2
import numpy as np

# 'path to input image/video'

# 'path to yolo config file' 
# download https://github.com/arunponnusamy/object-detection-opencv/blob/master/yolov3.cfg
CONFIG='./yolov3.cfg'

# 'path to text file containing class names'
# download https://github.com/arunponnusamy/object-detection-opencv/blob/master/yolov3.txt
CLASSES='./yolov3.txt'

# 'path to yolo pre-trained weights' 
# wget https://pjreddie.com/media/files/yolov3.weights
WEIGHTS='./yolov3.weights'

In [2]:
import os  
print(os.path.exists(CLASSES))
print(os.path.exists(CONFIG))
print(os.path.exists(WEIGHTS))

True
True
True


In [3]:
classes = None
with open(CLASSES, 'r') as f:
     classes = [line.strip() for line in f.readlines()]
        
scale = 0.00392
conf_threshold = 0.5
nms_threshold = 0.4

# generate different colors for different classes 
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

In [4]:
def get_output_layers(net): 
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

# function to draw bounding box on the detected object with class name
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):
    label = str(classes[class_id])
    color = COLORS[class_id]
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)
    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [5]:
def processImage(image, index):

    Width = image.shape[1]
    Height = image.shape[0]

    # read pre-trained model and config file
    net = cv2.dnn.readNet(WEIGHTS, CONFIG)

    # create input blob 
    blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)
    # set input blob for the network
    net.setInput(blob)

    # run inference through the network
    # and gather predictions from output layers
    outs = net.forward(get_output_layers(net))

    # initialization
    class_ids = []
    confidences = []
    boxes = []
    # for each detetion from each output layer 
    # get the confidence, class id, bounding box params
    # and ignore weak detections (confidence < 0.5)
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])
                    
    # apply non-max suppression
    
    classified_objects = []
    for i in class_ids:
        classified_objects.append(classes[i])
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    
    # go through the detections remaining
    # after nms and draw bounding box
    for i in indices:
        i = i[0]
        box = boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
    
        draw_bounding_box(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))
    
    # display output image    
    
    #cv2.imshow(out_image_name, image)
    # wait until any key is pressed
    #cv2.waitKey()
    # save output image to disk
    return classified_objects

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

directory = 'blondeblissblog'
pr_classes = {}
for filename in os.listdir(directory):
    if filename.endswith('.jpg'):
        print(directory + '/' + filename)
        temp = mpimg.imread(directory + '/' + filename)
        pr_classes[filename] = processImage(temp, filename)

blondeblissblog/52681242_394460821132229_7190371444573405800_n.jpg
blondeblissblog/44559289_422054221659752_681395479586625320_n.jpg
blondeblissblog/27577301_2011918939076075_8327663294538055680_n.jpg
blondeblissblog/37587258_454793538359008_50633983632867328_n.jpg
blondeblissblog/18646388_878870025586977_4433690251419975680_n.jpg
blondeblissblog/21294544_119293262021495_4526072184753356800_n.jpg
blondeblissblog/29403007_188741655187154_308275521098088448_n.jpg
blondeblissblog/35484217_469251206868384_7036994178634481664_n.jpg
blondeblissblog/41052389_282946902545445_3718586718466570339_n.jpg
blondeblissblog/19380035_588696981518478_1092235450665402368_n.jpg
blondeblissblog/28156459_1544994992265590_9051607885141770240_n.jpg
blondeblissblog/18444933_139416959936150_2097495331095707648_n.jpg
blondeblissblog/19985770_1134152103395474_5923222297363087360_n.jpg
blondeblissblog/43778667_160861991535292_7147616867899903623_n.jpg
blondeblissblog/41984928_167165774195619_7749818069722388412_n.

In [12]:
import csv
w = csv.writer(open("kylie_output.csv", "w"))
for key, val in pr_classes.items():
    w.writerow([key, val])